In [4]:
import numpy as np
import pandas as pd

def kabsch(P, Q):
    """Computes the optimal rotation and translation using the Kabsch algorithm."""
    centroid_P = np.mean(P, axis=1, keepdims=True)
    centroid_Q = np.mean(Q, axis=1, keepdims=True)
    
    P_centered = P - centroid_P
    Q_centered = Q - centroid_Q
    
    H = P_centered @ Q_centered.T
    U, _, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T
    
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = Vt.T @ U.T
    
    T = centroid_Q - R @ centroid_P
    
    error = np.linalg.norm(Q - (R @ P + T)) / np.sqrt(P.shape[1])
    return R, T.flatten(), error

# Local coordinates
xyz = np.array([[-1.3276, 14.2143, -1.8844],
                [-1.3719, 15.6844, 1.3844],
                [-1.3014, 13.3857, 2.952],
                [3.1823, -9.3287, -1.301],
                [3.0421, -7.2276, 1.0061],
                [3.207, -9.6959, 4.2199],
                [1.4108, 8.0448, 1.7928],
                [0, 0, 0]])

# RD coordinates (measured), adjusted
xyz_RD_measured = np.array([[72488.47, 452098.15, 5.45],
                             [72487.617, 452094.633, 5.505],
                             [72484.851, 452094.844, 5.324],
                             [72469.414, 452112.26, 1.774],
                             [72469.66, 452109.102, 1.841],
                             [72465.737, 452108.107, 1.706],
                             [72481.243, 452099.093, 2.879],
                             [72476.073, 452105.429, 4.692]]) - np.array([72000, 452000, 0])

# CloudCompare transformation matrix
transformation_matrix_CC = np.array([[-0.019893104, 0.789837182, -0.612993896, 476.05029],
                                     [0.032912239, -0.612265587, -0.789966881, 105.4001],
                                     [-0.999260247, -0.035889894, -0.013815446, 4.627503872]])

rotation_matrix_CC = transformation_matrix_CC[:, :3]
translation_matrix_CC = transformation_matrix_CC[:, 3]

# Compute transformation matrix using Kabsch algorithm
rotation_matrix_calc, translation_matrix_calc, lrms = kabsch(xyz.T, xyz_RD_measured.T)

# Compare transformation matrices
rotation_diff = rotation_matrix_CC - rotation_matrix_calc
translation_diff = translation_matrix_CC - translation_matrix_calc

# Calculate transformed coordinates
xyz_RD_calc = xyz @ rotation_matrix_calc.T + translation_matrix_calc

# Compute errors
error = xyz_RD_measured - xyz_RD_calc

distance = np.linalg.norm(error, axis=1)  # Total error per point in meters

# Compute RMS error
rms_error = np.sqrt(np.mean(distance**2))

# Create a Pandas DataFrame
columns = ['x local', 'y local', 'z local',
           'x RD_measured', 'y RD_measured', 'z RD_measured',
           'x RD_calculated', 'y RD_calculated', 'z RD_calculated',
           'error x', 'error y', 'error z', '|distance| [m]']

data = np.hstack((xyz, xyz_RD_measured, xyz_RD_calc, error, distance.reshape(-1, 1)))
table_all = pd.DataFrame(data, columns=columns)

# Display the table
print(table_all)
print(f"RMS Error: {rms_error:.6f} m")

# Optionally, save to a CSV file
# table_all.to_csv("transformation_results.csv", index=False)


   x local  y local  z local  x RD_measured  y RD_measured  z RD_measured  \
0  -1.3276  14.2143  -1.8844        488.470         98.150          5.450   
1  -1.3719  15.6844   1.3844        487.617         94.633          5.505   
2  -1.3014  13.3857   2.9520        484.851         94.844          5.324   
3   3.1823  -9.3287  -1.3010        469.414        112.260          1.774   
4   3.0421  -7.2276   1.0061        469.660        109.102          1.841   
5   3.2070  -9.6959   4.2199        465.737        108.107          1.706   
6   1.4108   8.0448   1.7928        481.243         99.093          2.879   
7   0.0000   0.0000   0.0000        476.073        105.429          4.692   

   x RD_calculated  y RD_calculated  z RD_calculated   error x   error y  \
0       488.458809        98.142040         5.470025  0.011191  0.007960   
1       487.617073        94.658248         5.416357 -0.000073 -0.025248   
2       484.839143        94.829633         5.406746  0.011857  0.014367   
3 

In [10]:
## Lidar 1 storm 2 


# Local coordinates
xyz = np.array([
                [0, 0, 0],
                [-1.0195, 13.20445, 0.569929],
                [-1.4134, 14.20554, 3.069763],
                [-1.1332, 13.1561, 5.332819],
                [3.079913, -10.7078, -1.39568],
                [2.991938, -8.98624, 0.407583],
                [3.190001, -11.6208, 3.051276],
                [1.604542, 4.579292, 2.149369]
            ])

# RD coordinates (measured), adjusted
xyz_RD_measured = np.array([
                [72492.06, 452127.3, 4.577],
                [72502.56, 452119.2, 5.333],
                [72501.93, 452116.6, 5.689],
                [72499.7, 452115.3, 5.54],
                [72484.24, 452134.8, 1.728],
                [72484.5, 452132.3, 1.824],
                [72480.8, 452131.7, 1.639],
                [72494.5, 452122.9, 2.81]
            ]) - np.array([72000, 452000, 0])

# CloudCompare transformation matrix
transformation_matrix_CC = np.array([[-0.019893104, 0.789837182, -0.612993896, 476.05029],
                                     [0.032912239, -0.612265587, -0.789966881, 105.4001],
                                     [-0.999260247, -0.035889894, -0.013815446, 4.627503872]])

rotation_matrix_CC = transformation_matrix_CC[:, :3]
translation_matrix_CC = transformation_matrix_CC[:, 3]

# Compute transformation matrix using Kabsch algorithm
rotation_matrix_calc, translation_matrix_calc, lrms = kabsch(xyz.T, xyz_RD_measured.T)

# Compare transformation matrices
rotation_diff = rotation_matrix_CC - rotation_matrix_calc
translation_diff = translation_matrix_CC - translation_matrix_calc

# Calculate transformed coordinates
xyz_RD_calc = xyz @ rotation_matrix_calc.T + translation_matrix_calc

# Compute errors
error = xyz_RD_measured - xyz_RD_calc

distance = np.linalg.norm(error, axis=1)  # Total error per point in meters

# Compute RMS error
rms_error = np.sqrt(np.mean(distance**2))

# Create a Pandas DataFrame
columns = ['x local', 'y local', 'z local',
           'x RD_measured', 'y RD_measured', 'z RD_measured',
           'x RD_calculated', 'y RD_calculated', 'z RD_calculated',
           'error x', 'error y', 'error z', '|distance| [m]']

data = np.hstack((xyz, xyz_RD_measured, xyz_RD_calc, error, distance.reshape(-1, 1)))
table_all = pd.DataFrame(data, columns=columns)

# Display the table
print(table_all)
print(f"RMS Error: {rms_error:.6f} m")

print()
print(rotation_matrix_calc)
print(translation_matrix_calc)

# Optionally, save to a CSV file
table_all.to_csv("transformation_results.csv", index=False)


    x local    y local   z local  x RD_measured  y RD_measured  z RD_measured  \
0  0.000000   0.000000  0.000000         492.06          127.3          4.577   
1 -1.019500  13.204450  0.569929         502.56          119.2          5.333   
2 -1.413400  14.205540  3.069763         501.93          116.6          5.689   
3 -1.133200  13.156100  5.332819         499.70          115.3          5.540   
4  3.079913 -10.707800 -1.395680         484.24          134.8          1.728   
5  2.991938  -8.986240  0.407583         484.50          132.3          1.824   
6  3.190001 -11.620800  3.051276         480.80          131.7          1.639   
7  1.604542   4.579292  2.149369         494.50          122.9          2.810   

   x RD_calculated  y RD_calculated  z RD_calculated   error x   error y  \
0       492.116804       127.355321         4.576915 -0.056804 -0.055321   
1       502.521496       119.175112         5.317367  0.038504  0.024888   
2       501.882812       116.559296       